In [ ]:
# ADVANCED FOREX TRADING BOT (High-Profit Strategy)import MetaTrader5 as mt5import pandas as pdimport numpy as npimport timefrom datetime import datetimefrom ta.momentum import RSIIndicator, StochasticOscillatorfrom ta.trend import MACDsymbols = [    "EURUSD", "GBPUSD", "USDJPY", "USDCHF", "USDCAD", "AUDUSD", "NZDUSD",    "EURGBP", "EURJPY", "EURCHF", "EURAUD", "EURNZD", "EURCAD",    "GBPJPY", "GBPCHF", "GBPCAD", "GBPAUD", "GBPNZD",    "AUDJPY", "AUDNZD", "AUDCAD", "AUDCHF",    "NZDJPY", "NZDCHF", "NZDCAD",    "CHFJPY", "CADJPY", "CADCHF",    "USDHKD", "USDSGD", "XAUUSD"]lot = 0.1sl_pips = 50tp_pips = 100timeframe = mt5.TIMEFRAME_M15if not mt5.initialize():    print("MT5 Init Failed")    mt5.shutdown()else:    print("MT5 Connected ✅")def get_data(symbol, timeframe, bars=150):    data = mt5.copy_rates_from_pos(symbol, timeframe, 0, bars)    df = pd.DataFrame(data)    df['time'] = pd.to_datetime(df['time'], unit='s')    df.rename(columns={        'open': 'Open', 'high': 'High', 'low': 'Low',        'close': 'Close', 'tick_volume': 'Volume'    }, inplace=True)    return df[['time', 'Open', 'High', 'Low', 'Close', 'Volume']]def apply_indicators(df):    df['RSI'] = RSIIndicator(df['Close'], window=14).rsi()    macd = MACD(df['Close'])    df['MACD'] = macd.macd()    df['MACD_Signal'] = macd.macd_signal()    df['EMA50'] = df['Close'].ewm(span=50).mean()    df['EMA200'] = df['Close'].ewm(span=200).mean()    stoch = StochasticOscillator(df['High'], df['Low'], df['Close'], window=14)    df['Stoch_K'] = stoch.stoch()    return dfdef detect_patterns(df):    df['Doji'] = np.abs(df['Open'] - df['Close']) <= (df['High'] - df['Low']) * 0.1    df['Engulfing'] = (df['Open'].shift(1) > df['Close'].shift(1)) & \                      (df['Open'] < df['Close']) & \                      (df['Open'] < df['Close'].shift(1)) & \                      (df['Close'] > df['Open'].shift(1))    df['Hammer'] = ((df['High'] - df['Low']) > 3 * np.abs(df['Open'] - df['Close'])) & \                   ((df['Close'] - df['Low']) / (.001 + df['High'] - df['Low']) > 0.6)    df['MorningStar'] = (        (df['Close'].shift(2) < df['Open'].shift(2)) &        (np.abs(df['Open'].shift(1) - df['Close'].shift(1)) < 0.001) &        (df['Close'] > df['Open']) & (df['Close'] > df['Open'].shift(1))    )    df['PiercingLine'] = (        (df['Close'].shift(1) < df['Open'].shift(1)) &        (df['Open'] < df['Low'].shift(1)) &        (df['Close'] > (df['Open'].shift(1) + df['Close'].shift(1)) / 2)    )    return dfdef get_trend_confirmation(symbol):    df = get_data(symbol, mt5.TIMEFRAME_H1, bars=100)    if df.empty: return False    ema = df['Close'].ewm(span=50).mean()    return df['Close'].iloc[-1] > ema.iloc[-1]def place_order(order_type, symbol):    tick = mt5.symbol_info_tick(symbol)    point = mt5.symbol_info(symbol).point    price = tick.ask if order_type == mt5.ORDER_TYPE_BUY else tick.bid    sl = price - sl_pips * point if order_type == mt5.ORDER_TYPE_BUY else price + sl_pips * point    tp = price + tp_pips * point if order_type == mt5.ORDER_TYPE_BUY else price - tp_pips * point    request = {        "action": mt5.TRADE_ACTION_DEAL,        "symbol": symbol,        "volume": lot,        "type": order_type,        "price": price,        "sl": sl,        "tp": tp,        "deviation": 10,        "magic": 123456,        "comment": "ProfitMaxBot",        "type_time": mt5.ORDER_TIME_GTC,        "type_filling": mt5.ORDER_FILLING_IOC    }    result = mt5.order_send(request)    if result.retcode != mt5.TRADE_RETCODE_DONE:        print(f"❌ Order Failed for {symbol}: {result.retcode}")    else:        print(f"✅ Order Placed for {symbol}: {result.retcode}")def analyze_and_trade(symbol):    df = get_data(symbol, timeframe)    if df.empty or len(df) < 50:        print(f"Insufficient data for {symbol}")        return    df = apply_indicators(df)    df = detect_patterns(df)    latest = df.iloc[-1]    trend_ok = get_trend_confirmation(symbol)    pattern_match = latest[['Doji', 'Engulfing', 'Hammer', 'MorningStar', 'PiercingLine']].any()    oversold = latest['RSI'] < 40 and latest['MACD'] > latest['MACD_Signal']    overbought = latest['RSI'] > 60 and latest['MACD'] < latest['MACD_Signal']    if oversold and pattern_match and trend_ok:        print(f"🔼 BUY signal for {symbol}")        place_order(mt5.ORDER_TYPE_BUY, symbol)    elif overbought and pattern_match and not trend_ok:        print(f"🔽 SELL signal for {symbol}")        place_order(mt5.ORDER_TYPE_SELL, symbol)    else:        print(f"⏸️ No clear signal for {symbol}")# Main Execution Loopwhile True:    for sym in symbols:        mt5.symbol_select(sym, True)        analyze_and_trade(sym)    print(f"\nCycle complete at {datetime.now()}. Sleeping 5 minutes...\n")    time.sleep(300)